In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

StatementMeta(, e99cc4c4-4695-4b62-9287-96243a7d3b40, 7, Finished, Available, Finished)

In [6]:
df = spark.sql("SELECT * FROM LH_Bronze.cleandata")
display(df)

StatementMeta(, e99cc4c4-4695-4b62-9287-96243a7d3b40, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ef68c5b9-b855-40d3-a036-dc16ad112bdf)

In [7]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

model = (AnalyzeText()
            .setTextCol("description").setKind('SentimentAnalysis').setErrorCol('Error').setOutputCol('Response')
        )


StatementMeta(, e99cc4c4-4695-4b62-9287-96243a7d3b40, 9, Finished, Available, Finished)

In [8]:
result = model.transform(df)
display(result)

StatementMeta(, e99cc4c4-4695-4b62-9287-96243a7d3b40, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6688c077-ca23-429f-8f21-00b9af0a7f09)

In [9]:
sentiment = result.withColumn("Sentimentresult", result['Response']['documents']['sentiment'])
sentiment = sentiment.drop('Response','Error')
display(sentiment)

StatementMeta(, e99cc4c4-4695-4b62-9287-96243a7d3b40, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9aae4a76-e838-4716-99b1-e4294b354a7a)

In [10]:
from pyspark.sql.utils import *

#sentiment.write.format('delta').saveAsTable('analysedData')

try:

    tablename = 'analysedData'

    sentiment.write.format('delta').saveAsTable('tablename')

except AnalysisException:

    print('Table Already Exists')

    sentiment.createOrReplaceTempView("vw_analysedData")

    spark.sql(f""" MERGE INTO {tablename} target
                   USING vw_analysedData source
                   ON target.url = source.url


                   WHEN MATCHED AND 
                   target.name <> source.name OR
                   target.description <> source.description OR
                   target.published <> source.published OR
                   target.Sentimentresult <> source.Sentimentresult

                   THEN UPDATE SET *


                   WHEN NOT MATCHED 
                   THEN INSERT *    
    
    """)


StatementMeta(, e99cc4c4-4695-4b62-9287-96243a7d3b40, 12, Finished, Available, Finished)

Table Already Exists


In [11]:
%%sql

select count(*) from analysedData

StatementMeta(, e99cc4c4-4695-4b62-9287-96243a7d3b40, 13, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>